<img src='data/images/section-notebook-header.png' />

# Normalization

Normalization in the context of Natural Language Processing (NLP) refers to the process of transforming text data into a standard or canonical form, making it easier to handle and analyze. It involves removing or reducing variations that do not contribute to the overall meaning of the text.

Normalization techniques are applied to address different types of variations in text, including:

* **Case normalization:** Converting all characters to lowercase or uppercase. This helps to ensure that the same word in different cases is treated as the same word. For example, "apple" and "Apple" would be normalized to "apple."

* **Accent removal:** Removing diacritical marks or accents from characters. For instance, converting "résumé" to "resume" by removing the accent.

* **Lemmatization:** Reducing words to their base or root form, known as the lemma. This is typically achieved by applying linguistic rules or using language-specific dictionaries. For example, lemmatizing "running" would yield "run."

* **Stemming:** Reducing words to their stem, which is a crude approximation of the root form. Stemming algorithms often rely on heuristic rules to remove prefixes and suffixes. For instance, stemming "running" would result in "run."

* **Stop word removal:** Eliminating common words that do not carry much semantic information, such as articles, prepositions, and conjunctions. Examples of stop words include "the," "and," and "in."

* **Punctuation removal:** Eliminating punctuation marks from the text. Punctuation is often not relevant for many NLP tasks and can be safely removed.

* **Numeric normalization:** Replacing numbers with a standardized representation, such as replacing specific numbers with a generic token like "NUM" or normalizing all numbers to a standard format.

Normalization is an essential preprocessing step in NLP tasks like text classification, information retrieval, sentiment analysis, and machine translation. It helps to reduce the vocabulary size, handle variations, improve generalization, and facilitate meaningful comparisons between text inputs.

In this notebook, we look at some of the basic normalization steps and cover Stemming and Lemmatization in a separate notebook.

## Setting up the Notebook

### Import all Required Packages

In [ ]:
import re

from nltk.corpus import stopwords

### Load Stopwords as Provided by NLTK

In [ ]:
nltk_stopwords = list(stopwords.words('english'))

print(nltk_stopwords)

### Define an Example Document

We first create a list of sentences, and we make it particularly "ugly".


In [ ]:
sentences = ["Text processing with Python    is great hahahaha.", 
             "It     isn't (very) cooomplicated to get started.",
             "However,careful to...you know....avoid mistakes.",
             "Contact me at vonderweth@nus.edu.sg; see http://nus.edu.sg.",
             "This is so cooool #nltkrocks     :))) :-P <3."]

---

## Normalize Raw Sentences

We first perform different normalization steps directly on the raw input sentences. In other words, we perform normalization for splitting the sentences into tokens using a tokenizer. There is generally no fundamental differences between these 2 approaches, however, for some steps the choice of order might make normalization more convenient. Throughout the example, thin about which normalization steps are more convenient to before or after tokenization.

#### Lowercase all letters

In [ ]:
for s in sentences:
    print(s.lower())

**Important:** Lowercasing all letters in a text for NLP can be problematic in certain situations due to the following reasons:

* **Ambiguity in proper nouns:** Lowercasing all letters removes the distinction between proper nouns and common nouns. Proper nouns, such as names of people, places, organizations, or specific entities, are often capitalized. By lowercasing them, we lose the information about the specificity or identity associated with these proper nouns. This can impact tasks like named entity recognition, entity disambiguation, or information retrieval, where the distinction between proper and common nouns is crucial.

* **Sentence-level sentiment analysis:** In sentiment analysis, the sentiment of a sentence can sometimes be affected by the capitalization of certain words. For example, consider the sentence "I love it" versus "I LOVE it." The capitalization of "LOVE" in the latter sentence indicates stronger positive sentiment compared to the former. By lowercasing all letters, this distinction is lost, and the sentiment analysis may not accurately capture the intensity of sentiment expressed.

* **Acronyms and abbreviations:** Lowercasing can also pose challenges when dealing with acronyms or abbreviations. Lowercasing these terms may lead to ambiguity or loss of their specific meaning. For instance, "USA" and "usa" can refer to different things when considering case sensitivity. By lowercasing all letters, we lose the information that "USA" is an acronym for "United States of America."

* **Text normalization requirements:** In some cases, text normalization requirements may call for preserving the original case of the text. For instance, in code snippets, URLs, or other technical contexts, case sensitivity is essential. Lowercasing all letters would change the intended meaning or functionality of the text.

While lowercasing can be a useful preprocessing step in many NLP tasks, it is important to consider the specific requirements of the task at hand. In certain cases, preserving the original case or applying more advanced techniques like named entity recognition can help overcome the limitations of lowercasing.

#### Remove duplicate whitespaces

Duplicate whitespace can often occur when words or other parts of a text have been removed; including stopwords (see below). Most modern tokenizers will handle duplicate whitespaces gracefully, i.e., simply ignore them. This means that a tokenizer will yield the same result for `"I like burgers."` and `"I 	like  burgers  	."` (i.e., the list of tokens `["I", "like", "burger", "."]`). However, removing duplicate whitespaces can be very useful to clean a modified text if it is supposed to be read by humans again.

In [ ]:
for s in sentences:
    print(re.sub(r'\s+', ' ', s))
    

#### Remove ellipses (...)

Particularly in social media, users use ellipses (`...`) to express a pause or hesitation or emphasis in their writing. This typically does not represent well-formed English, and it's typically recommended to clean the text by removing ellipses. We can very easily do this using RegEx.


In [ ]:
for s in sentences:
    print(re.sub(r'([.]){2,}', ' ', s))

Ellipses are just an example of parts in a text we might want to remove. Other examples could include URLs, hashtags, email addresses, and so on -- the exact type of content to be removed will generally depend on the application task. Also apart from removing parts of the text, we can also consider replacing them. For example, we might want to remove specific URLs but retain the knowledge about the presence of a URL. We can do this by replacing all URLs with a unique placeholder (e.g., `<URL>`). Again, we can do this using RegEx (see examples in the corresponding notebook).

#### Wrapping it all Together (with additional steps)

In practice, we often decide on the applied normalization steps based on our task and wrap them in function, and apply this function over all of our input before further processing or analysis. The method `normalize_sentence()` below shows an example for this. The method performs the steps outlined above but also adds some additional ones; the comments for each line describe the performed step.


In [ ]:
def normalize_sentence(s):
    s = s.lower()                                                   # Lowercase whole sentence
    s = re.sub(r'\s+', ' ', s)                                      # Remove duplicate whitespaces
    s = re.sub(r'([.]){2,}', ' ', s)                                # Remove ellipses ...
    s = re.sub(r'([\w.-]+)([,;])([\w.-]+)', '\g<1>\g<2> \g<3>', s)  # Add missing whitespace after , and ;
    s = re.sub(r'(.+)\1{2,}', '\g<1>\g<1>', s)                      # Reduce repeated sequences to 2
    s = re.sub(r'[\(\[].*?[\)\]]', ' ', s)                          # Remove all words in brackets
    s = re.sub(r'(n\'t)', ' not', s)                                # Resolve contraction "-n't"
    s = re.sub(r'\s+', ' ', s)                                      # Remove duplicate whitespaces (again)
    s = s.strip()                                                   # Remove trailing whitespaces
    return s
    
for s in sentences:
    s_norm = normalize_sentence(s)
    print(s_norm)

---

## Normalize Token Lists

Now we consider normalization after performing tokenization. We use the `TweetTokenizer` of NLTK since we have many informal tokens such as email addresses, URLs, and emoticons.

In [ ]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

### Tokenize all Sentences

Let's first tokenize each sentence. Here, we are not overly concerned about the quality of the tokenizer itself, and just use the output as presented for the subsequent normalization steps.

In [ ]:
token_lists = [ tweet_tokenizer.tokenize(s) for s in sentences ]

for tl in token_lists:
    print(tl)

### Normalize all Tokens

#### Lowercase all Letters

Instead of lowercasing all sentences as a whole, we now lowercase each individual token. Given that's a very simple task, the required code is still exceedingly simple.

In [ ]:
for tl in token_lists:
    print([token.lower() for token in tl])

Of course, the same considerations whether to lowercase or not still apply; see above.

#### Remove duplicate whitespaces

Well, not needed anymore. The tokenizer already took care of it.

#### Remove ellipses (...)

In contrast to using RegEx to remove ellipses from a long string forming a whole sentence, here their removal translates to removing all tokens of the form ".." ot with even more consecutive dots. This is arguably easier to write as code than coming up with the correct RegEx.

In [ ]:
for tl in token_lists:
    print([token for token in tl if not token.startswith("..")])

#### Wrapping it all Together (with additional steps)

As above, we can combine all the normalization steps we want to need to perform on each token in a method for easy and convenient re-use. The method `normalized_token_list()` performs the same steps as the method `normalize_sentence()` from above. Note that the main difference is that `normalized_token_list()` now has a loop that iterates all tokens of a token list (i.e., the output of the tokenizer for a sentence).

In [ ]:
def normalized_token_list(token_list):
    for j, token in enumerate(token_list):
        s = token
        s = s.lower()
        s = re.sub(r'([.]){2,}', ' ', s)                                # Remove ellipses ...
        s = re.sub(r'([\w.-]+)([,;])([\w.-]+)', '\g<1>\g<2> \g<3>', s)  # Add missing whitespace after , and ;
        s = re.sub(r'(.+)\1{2,}', '\g<1>\g<1>', s)                      # Reduce repeated sequences to 2
        s = re.sub(r'[\(\[].*?[\)\]]', ' ', s)                          # Remove all words in brackets
        s = re.sub(r'(n\'t)', ' not', s)                                # Resolve contraction "-n't"
        s = re.sub(r'\s+', ' ', s)                                      # Remove duplicate whitespaces (again)
        s = s.strip()                                                   # Remove trailing whitespaces
        token_list[j] = s

    return token_list
    
for token_list in token_lists:
    token_list = normalized_token_list(token_list)
    print(token_list)

**Important:**

* Note how the conversion from "isn't" to "is not'' makes the result no longer perfectly tokenized. This can me easily remedied (a) by performing such normalization steps that might split a token into multiple tokens before the tokenization step, or (b) by performing a subsequent tokenization step within the normalization method for tokens that have been split.

* We also no longer correctly remove all words in parentheses, since the tokenizer split the opening and closing parentheses from the words. So either we perform this normalization step before tokenization, or we have to implement a more complex normalization method that keeps track of opening and closing parentheses and removes all tokens in between. The example code below briefly illustrates this idea:

In [ ]:
for i, tl in enumerate(token_lists):
    for j, token in enumerate(tl):
        s = token
        try:
            if tl[j-1] in "[(" and tl[j+1] in ")]":
                s = ''
                tl[j-1] = ''
                tl[j+1] = ''
        except:
            pass
        tl[j] = s
        
    token_lists[i] = [ token for token in tl if token.strip() != '' ]
    print(token_lists[i])

Note that this is a very simple code snippet and only works if there is only a single word in parentheses. The important take-away message here is that deciding when to normalize -- i.e., before or after tokenization -- is mainly a matter of convenience and the amount of code required to correctly implement all normalization steps.

### Remove Stopwords

In NLP, stopwords are commonly used words that are considered to have little or no semantic meaning and are often removed from text during preprocessing. These words are typically highly frequent and occur in almost every document or sentence, such as articles, conjunctions, prepositions, and common pronouns.

Examples of stopwords in English include words like "the," "and," "is," "in," "a," "an," "of," "to," "it," "that," and "as." These words are not specific to any particular topic or domain and are usually disregarded in NLP tasks because they do not contribute much to the overall meaning or information of the text.

Stopwords are removed from text data for various reasons, including:

* **Reducing noise:** By eliminating stopwords, we can focus on the more important words that carry the main content and meaning of the text.

* **Improving efficiency:** Stopword removal reduces the size of the text data, which can lead to faster processing and more efficient analysis.

* **Improving accuracy:** In certain NLP tasks, such as text classification or sentiment analysis, stopwords may not provide valuable information and can potentially introduce noise or bias into the analysis. Removing them can help improve the accuracy of such models.

It's important to note that the choice of stopwords can vary depending on the specific application or context. Some applications may consider additional words as stopwords, while others may retain certain stopwords if they are relevant for the analysis or task at hand.

Let's remove all the words/tokens from our example document that are part of the stopword list provided by NLTK. Not all packages or toolkits may use the exact same list of stopwords. Recall from the lecture that it might not be wise to remove all the stopwords. For example, removing stopwords such as *"not"* and *"n't"* can significantly affect a sentiment analysis solution since these words/tokens typically flip the sentiment of a following word or phrase.

In [ ]:
for i, tl in enumerate(token_lists):
    for j, token in enumerate(tl):
        if token in nltk_stopwords:
            token = ''
        tl[j] = token
    token_lists[i] = [ token for token in tl if token.strip() != '' ]
    
for token_list in token_lists:
    print(token_list)

**Important:** Stopword removal can be problematic for tasks such sentiment analysis. Stopwords, despite being common and often insignificant in terms of their individual meaning, can contribute valuable contextual information when it comes to sentiment analysis. Here are a few reasons why stopword removal can pose challenges for sentiment analysis:

* **Loss of context:** Stopwords, such as "not," "but," or "no," can significantly alter the meaning and sentiment of a sentence. Removing them might lead to the loss of negations or contrasting sentiments, resulting in an inaccurate sentiment analysis.

* **Emphasis and intensity:** Stopwords can convey emphasis and intensity in a sentence. For example, consider the sentence "This movie is not good." The word "not" in this context negates the positive sentiment conveyed by the word "good." Removing "not" would change the sentiment analysis result to positive, which is incorrect.

* **Domain-specific stopwords:** Some stopwords may be domain-specific and carry sentiment information relevant to a particular context. For instance, in the domain of customer reviews for hotels, words like "service" or "location" can carry significant sentiment clues. Removing them could result in the loss of valuable sentiment indicators.

* **Phrase-level sentiment:** Stopwords can be part of sentiment-bearing phrases or idioms. Removing stopwords might break down such phrases and prevent accurate sentiment analysis. For example, the phrase "cutting-edge technology" has a positive sentiment, but if the word "cutting" (a stopword) is removed, the sentiment may not be correctly captured.

While stopwords are often removed in NLP tasks to reduce noise and dimensionality, their removal for sentiment analysis should be approached with caution. It is essential to consider the specific requirements and characteristics of the sentiment analysis task and evaluate the impact of stopword removal on the accuracy and integrity of sentiment classification results. In some cases, keeping stopwords or applying more advanced techniques like part-of-speech tagging or dependency parsing can help preserve the sentiment-carrying information in the text.

---

## Summary

Choosing the appropriate normalization steps for natural language processing (NLP) tasks can be challenging due to several factors. Here is a brief summary of the challenges involved:

* **Task-specific requirements:** Different NLP tasks have varying requirements for text normalization. For instance, in sentiment analysis, preserving negations (e.g., "not good") is important, while in part-of-speech tagging, stemming or lemmatization may be preferred. Understanding the specific needs of the task is crucial when deciding on the normalization steps.

* **Language-specific considerations:** Different languages have unique linguistic characteristics, and the normalization techniques that work well for one language may not be applicable or effective for another. The challenges include language-specific morphology, compound words, inflections, and other linguistic phenomena. Developing language-specific normalization methods or leveraging language-specific resources becomes necessary.

* **Ambiguity and context:** Natural language is inherently ambiguous, and normalization can inadvertently alter the intended meaning. For example, the removal of stopwords can change the context of a sentence. Resolving ambiguities and maintaining contextual relevance while applying normalization steps can be complex and require careful consideration.

* **Trade-offs between precision and recall:** Normalization techniques can impact precision (accuracy of correctly normalized tokens) and recall (coverage of all relevant tokens). Aggressive normalization may improve precision but result in lower recall, while more relaxed normalization may achieve higher recall but sacrifice precision. Striking the right balance is crucial and depends on the specific NLP task and its objectives.

* **Out-of-vocabulary (OOV) terms:** Normalization steps such as stemming or lemmatization may encounter OOV terms not present in the normalization resources or models. Handling OOV terms requires robust fallback strategies to prevent loss of information or misinterpretation during the normalization process.

* **Performance and computational costs:** Some normalization techniques, such as rule-based methods or complex morphological analyzers, can be computationally expensive. The choice of normalization steps should consider the computational resources available, especially in scenarios with large-scale NLP tasks or real-time processing requirements.

Addressing these challenges requires a combination of linguistic expertise, domain knowledge, and experimentation to identify the most appropriate normalization steps for a given NLP task. It is crucial to evaluate and fine-tune normalization techniques to ensure they align with the objectives of the task while preserving the integrity and meaning of the text data.